In [1]:
import math
import matplotlib
import matplotlib.pyplot as plt
import random
random.seed(1)

In [2]:
#Calculando quantos bits são necessários para representar qualquer valor no dominio de -100 a 100
def bits_necessarios(dominio_max,dominio_min,precisao_casas_decimais):
    k_bits = math.log(((dominio_max - dominio_min) * 10 ** precisao_casas_decimais),2)
    #Precisamos ajustar(arredondar) a quantidade de bits para que seja inteiro
    if(k_bits > round(k_bits)):
        k_bits = round(k_bits) + 1
    return k_bits

In [3]:
#Definindo a precisão
def precisao(dominio_max,dominio_min,bits):
    precisao = (dominio_max - dominio_min) / ((2 ** bits) -1)
    return precisao

In [4]:
## Criando o Decodificador para representao o valor em binário para a representação no domínio em forma real
def decodificador_decimal(cadeia_gene_separada):
    decodificados_decimais = []
    for i in range(0,len(cadeia_gene_separada)):
        v1 = int(cadeia_gene_separada[i][0], 2)
        v2 = int(cadeia_gene_separada[i][1], 2)
        decodificados_decimais.append([v1,v2])
    return decodificados_decimais

In [5]:
## Realizaremos agora a conversão do objeto decimal, para o que é representado no domínio
def decodificador_real(decodificados_decimais,dominio_max,dominio_min,precisao_dominio):
    decodificados_reais = []
    for i in range(0,len(decodificados_decimais)):
        v1 = dominio_min + precisao_dominio * decodificados_decimais[i][0]
        v2 = dominio_min + precisao_dominio * decodificados_decimais[i][1]
        decodificados_reais.append([v1,v2])
    return decodificados_reais

In [6]:
def funcaoObjetiva(x,y):
    f6 = float(0.5 - (((math.sin(((x*x) + (y*y))**(1/2))) - 0.5) / (1 + 000.1 * ((x*x) + (y*y)))**2))
    return round(f6,10)

In [7]:
#Criando uma População Inicial Aleatória 50 bits 25 para x e 25 para y
def gerarPopulacao(individuos):
    cadeia_genetica_casal = []
    cont = 0
    while cont < individuos:
        caracteres = ""
        for cromossomo in range(0,50):
            caracteres = caracteres + str(random.randint(0,1))
        cadeia_genetica_casal.append(caracteres)
        cont = cont + 1
    return cadeia_genetica_casal

In [8]:
def geraPopulacaoSeparada(individuos,populacao):
    cadeia_gene_separada = []
    for i in range(0,len(populacao)):
        x = populacao[i][:25]
        y = populacao[i][25:]
        cadeia_gene_seperada_eixos = [x,y]
        cadeia_gene_separada.append(cadeia_gene_seperada_eixos)
    return cadeia_gene_separada

In [9]:
#Vamos aplicar a decodificação dos valores binários para os valores reais para podermos futuramente calcular o fitnnes
def decodificarPopulacao(cadeia_gene_separada,dominio_max,dominio_min,precisao_dominio):
    dec_decimais = decodificador_decimal(cadeia_gene_separada)
    dec_reais = decodificador_real(dec_decimais,dominio_max,dominio_min,precisao_dominio)
    return dec_reais

In [10]:
def calculandoAptidao(populacao_convertida):
    aptidoes = []
    for individuo in range(0,len(populacao_convertida)):
        aptidao = funcaoObjetiva(populacao_convertida[individuo][0],populacao_convertida[individuo][1])
        aptidoes.append(aptidao)
    return aptidoes

In [11]:
#Aagora vamos realizar a partição para o métodod da roleta, dado a aptidão de cada um dos nossos indivíduos
def particaoPopulacao(aptidoes):
    proporcoes = []
    aptidoes_por_individuo = aptidoes
    soma_aptidioes = 0
    for individuo in range(0,len(aptidoes_por_individuo)):
        soma_aptidioes = soma_aptidioes + aptidoes_por_individuo[individuo]
    for individuo in range(0,len(aptidoes_por_individuo)):
        proporcao_individuo_atual = aptidoes_por_individuo[individuo] / soma_aptidioes
        proporcoes.append(proporcao_individuo_atual)
    return proporcoes

In [12]:
## Vamos construir a roleta
def criandoRoleta(probabilidades_relativas):
    roleta = []
    soma_acumulativa = 0
    acumulado = 0
    for i in range(0,len(probabilidades_relativas)):
        soma_acumulativa = soma_acumulativa + probabilidades_relativas[i] #Soma de todas as probabilidades 
    for probabilidade in probabilidades_relativas:
        acumulado = acumulado + (probabilidade/soma_acumulativa)
        roleta.append(acumulado)
    return roleta

In [13]:
##def girandoRoleta(roleta,num_selecionados):
def girandoRoleta(roleta, quantidade_individuos):
    individuos_selecionados = []
    for i in range(0,quantidade_individuos):
        ##Giro a roleta
        giro_aleatorio = random.random()
        #Selecionando ddeposi que a roleta gira
        for i in range(len(roleta)):
            if roleta[i] >= giro_aleatorio:
                individuos_selecionados.append(i)
                break
    return individuos_selecionados

In [14]:
def reproducao(individuos_selecionados,taxa_cruzamento):
    ###### Defino a taxa de cruzamento para os pai ######
    taxa_primeira_parte_do_filho = round((taxa_cruzamento * 50) / 100) ## 50 quantidade de bits
    taxa_segunda_parte_do_filho = 50 - taxa_primeira_parte_do_filho ## Quantos bits serão usados no segundo corte
    ###### Cria e Armazeno os individuos selecionados para ser usado no laço ######
    pais = []
    pais.append(populacao[individuos_selecionados[0]]) #Pai_01
    pais.append(populacao[individuos_selecionados[1]])  #Pai_02
    pontos_corte = []
    #Pont0s de corte de cada pai, para seus filhos
    ponto_corte_pai01 = random.randint(0,50) #Para fins de teste escolha 10 E 40
    ponto_corte_pai02 = random.randint(0,50) #Para fins de teste escolha 10 E 40
    # Mostra os pontos de Corte de cada pai
    ##print("Pai1: {} \nPai2 :{}".format(ponto_corte_pai01,ponto_corte_pai02))
    #print("Pai 1 {} \nPai                                           {}".format(pai_01,pai_01[40:]))
    #print("Pai 2 {} \nPai             {}".format(pai_02,pai_02[10:]))
    ###### Armazeno os ponto de corte para ser usado no laço ######
    pontos_corte.append(ponto_corte_pai01)
    pontos_corte.append(ponto_corte_pai02)
    ###### Cria um lista contendo os genes dos pais separados, dado o ponto de corte que eels receberam ######
    filhos_nao_cruzados = []
    ###### Indice de acesso a minha lista de pontos de corte ######
    ponto = 0
    ###### Atribuo a partir do ponto de corte de cada um dos pais, seus respectivos cromossomos os filho nãa cruzados ######
    for pai in range(0,len(pais)):
        #print("############################################# {} CICLO #############################################".format(pai))
        #print("############################################# Ponto de Corte {} = {}  ##############################".format(ponto,pontos_corte[ponto]))
        ## 38 Primeiro + 12 Primeiro -> 38 Segundo + 12 Segundo ....... 
        ######################################  38 Primeiro ##########################################################################################
        ## Para o primeiro Filho - Genes do Pai 01 
        #Se de onde eu começo bater até o limite, tá ok ainda, posso pegar so os bits do vetor mesmo
        if len(pais[pai][pontos_corte[ponto]:]) >= taxa_primeira_parte_do_filho: # Se Do indice que eu to até o fim for maior que meu limite de 38 bits 75%
            parte_01_filho = pais[pai][pontos_corte[ponto]:taxa_primeira_parte_do_filho + pontos_corte[ponto]] # Meu filho pega só os 37 que lhe cabem
            ##PRINTS ABAIXO APENAS PARA MOSTRAR OS CASOS EM QUE SÃO 10 E 40
            #print("Filho Direto")
            #print("Pai {}           {} \nPT1P1                     {}".format(pai,pais[pai],pais[pai][pontos_corte[ponto]:]))
            #print("Filho01                   {}".format(parte_01_filho))
            #print(f"Primeira parte do filho/Bits " + str(len(parte_01_filho)))
        else:
            #print("Filho particionado")
            parte_01_pai = pais[pai][pontos_corte[ponto]:]
            parte_02_pai = pais[pai][:taxa_primeira_parte_do_filho - len(pais[pai][pontos_corte[ponto]:])]
            parte_01_filho = parte_01_pai + parte_02_pai
            ##PRINTS ABAIXO APENAS PARA MOSTRAR OS CASOS EM QUE SÃO 10 E 40
            #print("Filho particionado")
            #print("Pai {}          {}".format(pai,pais[pai]))
            #print("PT1P1                                                  {}".format(parte_01_pai))
            #print("PT2P2          {}".format(parte_02_pai))
            #print("Filho          {}".format(parte_01_filho))
            #print(f"Primeira parte do Filho/Bits " + str(len(parte_01_filho)))
        ######################################  12 Primeiro ##########################################################################################
        ## Para o primeiro Filho - Genes do Pai 01
        #Se de onde eu começo bater até o limite, tá ok ainda, posso pegar so os bits do vetor mesmo
        if len(pais[pai][pontos_corte[ponto]:]) >= taxa_segunda_parte_do_filho: # Se Do indice que eu to até o fim for maior que meu limite de 38 bits 75%
            parte_02_filho = pais[pai][pontos_corte[ponto]:taxa_segunda_parte_do_filho + pontos_corte[ponto]] # Meu filho pega só os 37 que lhe cabem
            ##PRINTS ABAIXO APENAS PARA MOSTRAR OS CASOS EM QUE SÃO 10 E 40
            #print("Filho Direto")
            #print("Pai {}           {} \nPT1P1                     {}".format(pai,pais[pai],pais[pai][pontos_corte[ponto]:]))
            #print("Filho01                   {}".format(parte_02_filho))
            #print(f"Primeira parte do filho/Bits " + str(len(parte_02_filho)))
        else:
            #print("Filho particionado")
            parte_01_pai = pais[pai][pontos_corte[ponto]:]
            parte_02_pai = pais[pai][:taxa_segunda_parte_do_filho - len(pais[pai][pontos_corte[ponto]:])]
            parte_02_filho = parte_01_pai + parte_02_pai
            ##PRINTS ABAIXO APENAS PARA MOSTRAR OS CASOS EM QUE SÃO 10 E 40
            #print("Filho particionado")
            #print("Pai {}          {}".format(pai,pais[pai]))
            #print("PT1P1                                                  {}".format(parte_01_pai))
            #print("PT2P2          {}".format(parte_02_pai))
            #print("Filho          {}".format(parte_02_filho))
            #print(f"Primeira parte do Filho/Bits " + str(len(parte_02_filho)))
        filhos_nao_cruzados.append([parte_01_filho,parte_02_filho])    
        ponto = ponto + 1
    return filhos_nao_cruzados

In [15]:
def crossover(filhos_sem_cruzamento):
    ###### Cria e Armazeno os individuos selecionados para ser usado no laço ######
    pais = []
    pais.append(populacao[individuos_selecionados[0]]) #Pai_01
    pais.append(populacao[individuos_selecionados[1]])  #Pai_02
    filhos_cruzados = []
    filho01 = filhos_sem_cruzamento[0][0] + filhos_sem_cruzamento[1][1]
    filho02 = filhos_sem_cruzamento[1][0] + filhos_sem_cruzamento[0][1]
    filhos_cruzados.append(filho01)
    filhos_cruzados.append(filho02)
    if filhos_cruzados[0] != pais[0] and filhos_cruzados[0] != pais[1] and filhos_cruzados[1] != pais[0] and filhos_cruzados[1] != pais[1] :
        return filhos_cruzados
       #print("ok")
    else:
        #print("Fodeu")
        novos_individuos_selecionados = girandoRoleta(roleta,2)
        nova_reproducao = reproducao(novos_individuos_selecionados,taxa_cruzamento)
        return crossover(nova_reproducao)

In [16]:
def mutacao(filhos_cruzados, taxa_mutacao):
    filhos_mutados = []
    for filho in filhos_cruzados:
        filho_mutado = ""
        for bit in filho:
            if random.random() < taxa_mutacao:
                # Com uma certa probabilidade (taxa_mutacao), o bit será invertido
                filho_mutado += "1" if bit == "0" else "0"
            else:
                filho_mutado += bit
        filhos_mutados.append(filho_mutado)
    return filhos_mutados

In [17]:
def pioresPaisMelhoresFilhos(populacao, filhos_mutados):
    # Decodificar a população atual
    populacao_convertida = decodificarPopulacao(populacao,dominio_max,dominio_min,precisao_dominio)
    # Calcular as aptidões dos indivíduos na população atual
    aptidoes = calculandoAptidao(populacao_convertida)
    # Encontrar os índices dos dois piores indivíduos na população atual
    indices_piores = sorted(range(len(aptidoes)), key=lambda i: aptidoes[i])[:2]
    # Calculaas aptidões dos filhos mutados
    aptidoes_filhos_mutados = calculandoAptidao(decodificarPopulacao(filhos_mutados,dominio_max,dominio_min,precisao_dominio))
    # Verificar se algum filho mutado é melhor que os piores indivíduos e troca
    for i, indice_pior in enumerate(indices_piores):
        if aptidoes_filhos_mutados[i] > aptidoes[indice_pior]:
            populacao[indice_pior] = filhos_mutados[i]
    return populacao

In [18]:
def novaGeracao(populacao, taxa_cruzamento, taxa_mutacao):
    # Partição da população atual para seleção
    probabilidades_relativas = particaoPopulacao(populacao)
    # Criação da roleta
    roleta = criandoRoleta(probabilidades_relativas)
    # Seleção de indivíduos para reprodução
    individuos_selecionados = girandoRoleta(roleta, 2)
    # Reprodução
    filhos_sem_cruzamento = reproducao(individuos_selecionados, taxa_cruzamento)
    # Crossover
    filhos_cruzados = crossover(filhos_sem_cruzamento)
    # Mutação
    filhos_mutados = mutacao(filhos_cruzados, taxa_mutacao)
    # Substituição dos piores pais pelos melhores filhos
    populacao = pioresPaisMelhoresFilhos(populacao, filhos_mutados)
    
    return populacao

In [59]:
#def algoritmoGenetico():
t = 0
individuos = 100
NGEN = 1
taxa_mutacao = 0.01  #Taxa de mutação de 1%
taxa_de_cruzamento = 75 ## 75% 
precisao_casas_decimais = 5 ## Precisão de casas decimais
dominio_max = 100
dominio_min = -100
## Hiper Parametros ##
bits = bits_necessarios(dominio_max,dominio_min,precisao_casas_decimais)
precisao_dominio = precisao(dominio_max,dominio_min,bits)
##Iniciando o Ag ##
populacao = gerarPopulacao(individuos)
populacao_separada = geraPopulacaoSeparada(individuos,populacao) ## Etapa para avaliacao
populacao_decodificada = decodificarPopulacao(populacao_separada,dominio_max,dominio_min,precisao_dominio) ## Etapa para avaliacao
## Realizo a avalização de toda a População de Indivíduos  - Aptidão relativa ##
aptidoes = calculandoAptidao(populacao_decodificada)
apitidoes_relativas = particaoPopulacao(aptidoes)
#Inicio as rodadas de Gerações (Seleção,Mutação,Cruzamento)
contador = 0
aptidao_alvo = 0.8
while contador <= NGEN:
    contador = contador + 1
    if max(aptidoes) >= aptidao_alvo:
        print(f'Solução ótima alcançada na geração {contador}')
        break
    roleta = criandoRoleta(apitidoes_relativas)
    ## Seleção de indivíduos para reprodução ##
    individuos_selecionados = girandoRoleta(roleta, 2)
    ## Reprodução (Cruzamento dos selecionados) ##
    filhos_sem_cruzamento = reproducao(individuos_selecionados,taxa_de_cruzamento)
    filhos_cruzados = crossover(filhos_sem_cruzamento)
    ## Mutação ##
    filhos_mutados = mutacao(filhos_cruzados, taxa_mutacao)
    ## Substituição ##
    populacao = pioresPaisMelhoresFilhos(populacao, filhos_mutados)
    ## Reavaliação da população ##
    populacao_decodificada_substituida = decodificarPopulacao(populacao, dominio_max, dominio_min, precisao_dominio)
    aptidoes_apos_substituicao = calculandoAptidao(populacao_decodificada)
    proporcoes_apos_substituicao = particaoPopulacao(aptidoes)
    print(f'Geração {contador}: Melhor aptidão = {max(aptidoes)}')

Geração 1: Melhor aptidão = 0.5003275415
Geração 2: Melhor aptidão = 0.5003275415


In [60]:
populacao_decodificada

[[41.929428038878086, -21.14706996521562],
 [-92.64512040153505, 95.70782171809142],
 [39.588932382730604, 60.79267742611995],
 [-34.38921971288978, 58.33471889301296],
 [-30.39808661932011, 28.632442016376302],
 [67.02631613690602, -2.9696256807335004],
 [-80.20377696167758, 33.206061518372934],
 [-19.424692375203747, -72.5143722449056],
 [-33.72759025477143, -29.635588217842226],
 [44.83339026073784, -46.03631931651591],
 [-55.62854873027053, -39.56053076864871],
 [-9.575823234791258, 44.4889588501739],
 [53.763352446656, 85.81122713718494],
 [7.947162030552676, -23.7366057555856],
 [58.407835912938, 59.71380352121005],
 [-64.54222692675074, -21.539399669748533],
 [-30.07438570482688, -87.08845934535442],
 [-39.36216054445984, 67.23241708375267],
 [19.92217659718324, -95.72892772343539],
 [11.671445121510175, -21.688220551258937],
 [-18.258408256125705, -65.5629863012727],
 [-10.626265723295987, -45.61546282814332],
 [-99.66277479120417, 79.55188690280576],
 [-82.5479502245173, -25.0

In [61]:
aptidoes

[0.5000068712,
 0.4999998569,
 0.5000028271,
 0.5000070323,
 0.5000420967,
 0.5000068764,
 0.5000024876,
 0.5000025772,
 0.4999928627,
 0.499997141,
 0.5000057506,
 0.4999884774,
 0.4999998406,
 0.5001482859,
 0.4999990542,
 0.5000064056,
 0.5000018783,
 0.4999997537,
 0.5000009615,
 0.5002584074,
 0.5000063627,
 0.5000044656,
 0.4999998264,
 0.5000026873,
 0.5000026857,
 0.500004498,
 0.5000004924,
 0.5000157537,
 0.5000000438,
 0.5000258536,
 0.4999992388,
 0.5003275415,
 0.5000009624,
 0.5000020269,
 0.4999957496,
 0.500001555,
 0.5000175425,
 0.5000000907,
 0.5000004268,
 0.4999996694,
 0.4913063717,
 0.5000163336,
 0.4999994787,
 0.5000046004,
 0.5000191092,
 0.5000014199,
 0.5000114537,
 0.4999963605,
 0.5000003819,
 0.4999510915,
 0.4999934334,
 0.4999993585,
 0.500005026,
 0.5000716364,
 0.5000034692,
 0.5000068508,
 0.4999994473,
 0.5000098056,
 0.5000001604,
 0.4999903982,
 0.5000015624,
 0.5000003325,
 0.499999978,
 0.5000108799,
 0.5000006119,
 0.4999998388,
 0.5000036281,


In [62]:
# Decodificar a população atual
#populacao_convertida = decodificarPopulacao(populacao,dominio_max,dominio_min,precisao_dominio)
# Calcular as aptidões dos indivíduos na população atual
#aptidoes = calculandoAptidao(populacao_convertida)
# Encontrar os índices dos dois piores indivíduos na população atual
#indices_piores = sorted(range(len(aptidoes)), key=lambda i: aptidoes[i])[:2]
# Calculaas aptidões dos filhos mutados
#aptidoes_filhos_mutados = calculandoAptidao(decodificarPopulacao(filhos_mutados,dominio_max,dominio_min,precisao_dominio))
# Verificar se algum filho mutado é melhor que os piores indivíduos e troca
#for i, indice_pior in enumerate(indices_piores):
#    if aptidoes_filhos_mutados[i] > aptidoes[indice_pior]:
#        populacao[indice_pior] = filhos_mutados[i]

In [63]:
sorted(aptidoes)[:2]

[0.4913063717, 0.4999510915]

In [64]:
filhos_mutados

['01001001000101110010011111010010010100101101101100',
 '10110110110000100100000100000000110100010110010001']

In [65]:
populacao_separada_filhos = geraPopulacaoSeparada(2,filhos_mutados) ## Etapa para avaliacao
populacao_decodificada_filhos = decodificarPopulacao(populacao_separada_filhos,dominio_max,dominio_min,precisao_dominio) ## Etapa para avaliacao
aptidoes_filhos_mutados = calculandoAptidao(populacao_decodificada_filhos)

In [66]:
populacao_separada_filhos

[['0100100100010111001001111', '1010010010100101101101100'],
 ['1011011011000010010000010', '0000000110100010110010001']]

In [67]:
populacao_decodificada_filhos

[[-42.898086991849155, 28.630713481626316],
 [42.78032013119221, -98.72197505003139]]

In [68]:
aptidoes_filhos_mutados

[0.4999934628, 0.4999998494]

In [69]:
sorted(aptidoes)[:2]

[0.4913063717, 0.4999510915]

In [82]:
min(aptidoes)
nova_lista = aptidoes
len(aptidoes)

98

In [68]:
if aptidoes_filhos_mutados[0] > apitdoes_ordenadas_populacao[0]:
    apitdoes_ordenadas_populacao[0] = aptidoes_filhos_mutados[0]
    print("Ok")
if aptidoes_filhos_mutados[1] > apitdoes_ordenadas_populacao[1]:
    apitdoes_ordenadas_populacao[1] = aptidoes_filhos_mutados[1]
    print("Ok")

Ok
Ok
